## Machine learning on EEG data

This tutorial walks through some basic tools for applying ML techniques on EEG data through MNE

### 1. Importing and preprocessing data

In [1]:
import mne

# Folder & files containing the data:
data_path = '/Users/athina/Documents/Bremen2018/PracticalPrep/'
data_file = '817_1_PDDys_ODDBALL_Clean_curated'

filename = data_path + data_file

# We read the EEG epochs:
epochs = mne.read_epochs(filename + '.fif')

epochs = epochs['Standard', 'Novel']

epochs.filter(l_freq = 0.1, h_freq = 20)
epochs.apply_baseline((None, 0))

This filename (/Users/athina/Documents/Bremen2018/PracticalPrep/817_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
Reading /Users/athina/Documents/Bremen2018/PracticalPrep/817_1_PDDys_ODDBALL_Clean_curated.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
189 matching events found
No baseline correction applied


<ipython-input-1-892008160184>:10: RuntimeWarning: This filename (/Users/athina/Documents/Bremen2018/PracticalPrep/817_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(filename + '.fif')


189 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Setting up band-pass filter from 0.1 - 20 Hz
l_trans_bandwidth chosen to be 0.1 Hz
h_trans_bandwidth chosen to be 5.0 Hz
Filter length of 16501 samples (33.002 sec) selected
filter_length (16501) is longer than the signal (301), distortion is likely. Reduce filter length or filter a longer signal.


<ipython-input-1-892008160184>:14: RuntimeWarning: filter_length (16501) is longer than the signal (301), distortion is likely. Reduce filter length or filter a longer signal.
  epochs.filter(l_freq = 0.1, h_freq = 20)


Applying baseline correction (mode: mean)


<EpochsFIF  |   160 events (all good), -0.1 - 0.5 sec, baseline [None, 0], ~22.2 MB, data loaded,
 'Novel': 30
 'Standard': 130>

The main information we wish to retain is (a) the data (b) the true labels

In [2]:
# and get the data:
data = epochs._data

# and labels:
labels = epochs.events[:,-1]

##### Reminder exercise: how many trials of each condition do we have?

In [3]:
epochs.event_id

{'Standard': 201, 'Novel': 202}

### 2. Using sklearn to classify EEG data
We can now proceed with some mne.decoding and sklearn functionalities:

In [5]:
from mne.decoding import Vectorizer

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.model_selection import train_test_split

'make_pipeline' will create our analysis pipeline from a set of estimators, sklearn-style.
'Vectorizer' comes from MNE and will pull together all epochs x time x channels data to a vector of samples x channels
'StandardScaler' normalizes the data by their median

In [6]:
clf = make_pipeline(Vectorizer(), StandardScaler(),
                    svm.SVC(kernel='linear',C=1)
                   )

We can then use scikit-learn function 'train_test_split' to obtain a random split of training and test datasets:

In [7]:
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.4, random_state=0)

##### Exercise: What are the dimensions of the train / test datasets?

In [12]:
labels_train.shape

(96,)

Now we can treat 'clf' as any sklearn estimator, and fit the training data to the classifier:

In [13]:
clf.fit(data_train, labels_train)

Pipeline(memory=None,
     steps=[('vectorizer', <mne.decoding.transformer.Vectorizer object at 0x10d18e128>), ('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svc', SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

And we can then use the classifier to predic the labels of the test data:

In [14]:
clf.predict(data_train)

array([202, 202, 201, 201, 202, 202, 201, 201, 201, 202, 201, 201, 201,
       201, 201, 201, 201, 202, 201, 201, 201, 201, 201, 201, 201, 201,
       202, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 202,
       201, 201, 201, 201, 201, 201, 202, 201, 201, 202, 202, 201, 201,
       201, 202, 201, 201, 201, 201, 201, 201, 202, 201, 201, 201, 201,
       201, 201, 202, 201, 202, 201, 201, 201, 201, 201, 201, 202, 201,
       201, 201, 201, 202, 202, 201, 201, 202, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201])

Or, we can directly compute the classification score:

In [15]:
clf.score(data_train,labels_train)

1.0

##### Exercise: What is the classification score on the test dataset? What do you observe?

In [16]:
clf.score(data_test,labels_test)

0.859375

Alternatively, we can also compute the classification accuracy based on cross-validation. By default, the selected model will be fitted on 3 different splits of the data:

In [17]:
clf_CV = make_pipeline(Vectorizer(), StandardScaler(),
                    svm.SVC(kernel='linear',C=1)
                   )

cross_val_score(clf_CV, data, labels)

array([0.88888889, 0.83018868, 0.81132075])

We can change the number of CV folds by including an additional parameter, 'cv':

Remember that for integer or None inputs, if the estimator is a classifier and the labels are binary or multiclass, StratifiedKFold is used. In all other cases, KFold is used (i.e. the folds are made by preserving the percentage of samples for each class):

In [18]:
scores = cross_val_score(clf_CV, data, labels, cv=5)
print(scores)

[0.875   0.8125  0.8125  0.84375 0.8125 ]


And we can now compute the mean and 95% confidence interval of the classification accuracy:

In [19]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.83 (+/- 0.05)


##### Exercise: What happens to the mean accuracy if you increase the CV folds? If you decrease them?

In [20]:
scores = cross_val_score(clf_CV, data, labels, cv=10)
print(scores)

[0.875  0.875  0.875  0.75   0.875  0.8125 0.875  0.75   0.75   0.9375]


Additionally, we can evaluate multiple metrics for our models at once, like the time it takes to fit them, the train and test scores with the function 'cross_validate':

In [21]:
from sklearn.model_selection import cross_validate

cross_validate(clf, data, labels)

/Users/athina/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([0.24624586, 0.22556305, 0.22564721]),
 'score_time': array([0.07464194, 0.06872201, 0.06971765]),
 'test_score': array([0.88888889, 0.83018868, 0.81132075]),
 'train_score': array([1., 1., 1.])}

Now we will first apply PCA to our data and then classify them. What do you observe?

In [28]:
from mne.decoding import UnsupervisedSpatialFilter

from sklearn.decomposition import PCA

pca = UnsupervisedSpatialFilter(PCA(n_components = 30)) # we create an instance of PCA

In [29]:
pca_data = pca.fit_transform(data)

Now, we classify the PCA-transformed data:

In [30]:
cross_validate(clf, pca_data, labels)

/Users/athina/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([0.14005566, 0.13438702, 0.13831019]),
 'score_time': array([0.04396009, 0.044554  , 0.04136968]),
 'test_score': array([0.87037037, 0.83018868, 0.77358491]),
 'train_score': array([1., 1., 1.])}

What do you observe?

### 3. Optimizing our classifier

In the above paradigm we just used a default implementation of SVM with a linear kernel and a hyperparameter C set to 1. Now we will estimate the parameters of SVM through exhaustive Grid Search.

'GridSearchCV' exhaustively tests candidate models from a grid of parameter values specified in the following form:

In [31]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}

Now we can use GridSearchCV directly in 'make_pipeline' to proceed with an optimized model:

In [32]:
clf_opt2 = make_pipeline(Vectorizer(), StandardScaler(), GridSearchCV(svm.SVC(), parameters,cv=5))
clf_opt2.fit(data, labels)
cross_val_score(clf_opt2,data,labels)

array([0.88888889, 0.83018868, 0.81132075])

What do you observe?

You can assess the optimized parameters through:

In [33]:
clf_opt2.steps[-1]

('gridsearchcv', GridSearchCV(cv=5, error_score='raise',
        estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
        fit_params=None, iid=True, n_jobs=1,
        param_grid={'kernel': ('linear', 'rbf'), 'C': [1, 10]},
        pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
        scoring=None, verbose=0))

The classifier that you trained & optimized above may still be biased. Any ideas why?

##### Exercise: how could you structure your dataset to make it unbiased?

In [34]:
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.5, random_state=0)

In [35]:
clf_opt2 = make_pipeline(Vectorizer(), StandardScaler(), GridSearchCV(svm.SVC(), parameters,cv=5))
clf_opt2.fit(data_train, labels_train)

Pipeline(memory=None,
     steps=[('vectorizer', <mne.decoding.transformer.Vectorizer object at 0x1095490f0>), ('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('gridsearchcv', GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decis...   pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0))])

In [39]:
import numpy as np

clf_opt2.steps[-1]
perf = cross_val_score(clf_opt2,data_test,labels_test, cv = 5)
np.mean(perf)

0.8004411764705882

### 4. Making a more sophisticated pipeline

In the steps above we were still evaluating a rather simplified version of our data, where we considered EEG data of one condition and the other irrespective of time. Now we will take the temporal dimension into account as well and will evaluate how classification accuracy changes across time.

For this we will use 'SlidingEstimator' from MNE:

In [40]:
from mne.decoding import SlidingEstimator, cross_val_multiscore

clf = make_pipeline(Vectorizer(), StandardScaler(),
                    svm.SVC(kernel='linear',C=1)
                   )

sl = SlidingEstimator(clf) # we apply the sliding estimator to 'clf'

CV_score_time = cross_val_multiscore(sl, data, labels)

[........................................] 100.00% Fitting SlidingEstimator |  
[........................................] 100.00% Fitting SlidingEstimator |  
[........................................] 100.00% Fitting SlidingEstimator |  


What are the dimensions of the classification score now? Why?

In [41]:
CV_score_time.shape

(3, 301)

We can now plot the classification accuracy across time:

In [43]:
import matplotlib.pyplot as plt

%matplotlib tk

fig, ax = plt.subplots()
ax.plot(epochs.times, CV_score_time.T)
plt.show()

##### Exercise: Can you now plot the mean classification accuracy over the CV folds as a function of time?

In [48]:
fig, ax = plt.subplots()
ax.plot(epochs.times, np.mean(CV_score_time,axis = 0))
plt.show()

Another important steps of classifying EEG data is to extract and visualize the features of our classifier. In the following case, we will extract the coefficients of SVM across time and plot them as topographic maps:

In [49]:
import numpy as np

topos = np.array([svm.SVC(kernel='linear').fit(time_point.T, labels).coef_ * time_point.std(1)
                  for time_point in data.T])[:, 0, :]
topo_ev = mne.EvokedArray(topos.T, info=epochs.info, tmin=-.1, nave=len(labels))
topo_ev.plot_joint(times=[.22, .3, .375, .45]);

##### Exercise: Can you now plot the classification coefficients on the time-points of maximal classification accuracy?

### 5. Implementing a more sophisticated pipeline

In the same pipeline we can additionally integrate dimensionality reduction techniques. How?